In [55]:
import sys

sys.path.append("../")

In [56]:
import pandas as pd
import spacy
from src.loader import TextLoader


In [57]:
nlp = spacy.load("en_core_web_lg")

In [58]:
loader = TextLoader()
df = loader.get_train_df()

In [59]:
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1622627660524,8,229,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1622627653021,230,312,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1622627671020,313,401,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1622627696365,402,758,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1622627759780,759,886,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...
...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1618153340639,2234,3203,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...
144289,4C471936CD75,1618153383399,3221,4509,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...
144290,4C471936CD75,1618024996127,4510,4570,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838
144291,4C471936CD75,1618025268756,4570,4922,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...


In [60]:
nlp.add_pipe("sentencizer")

In [61]:
unique_ids = df["id"].unique(); unique_ids

array(['423A1CA112E2', 'A8445CABFECE', '6B4F7A0165B9', ...,
       '408A7D3D2EEC', 'AFEC37C2D43F', '4C471936CD75'], dtype=object)

1. Iterate over tokens in a text
2. If the token is outside of a known discourse, mark it as the ND
3. If the token is starting a new discourse, mark it as the DS
4. If the token is continuing a discourse, mark it as the DB
5. If the token is ending a discourse, mark it as the DE

In [62]:
def format_embedded_discourse(discourse_text: str) -> str:
    discourse_text = "<DS>" + discourse_text
    if discourse_text.endswith("."):
        discourse_text = discourse_text[:-1] + "<DE>."
    else:
        discourse_text = discourse_text + "<DE>"

    return discourse_text

In [63]:
out_lines = []
for ind, unique_id in enumerate(unique_ids):
    text = loader.load_text_with_id(unique_id, purify_text=True, purify_discourses=True)
    print(text.text)
    print("\n----\n")

    embedded = text.text
    for disc_no, disc in enumerate(text.discourses):
        print(disc.discourse_text)
        print()
        fixed_start_ind = text.text.find(disc.discourse_text)
        if fixed_start_ind == -1:
            print("Could not find discourse text in text")
            raise

        fixed_end_ind = fixed_start_ind + len(disc.discourse_text)
        embedded = embedded[:fixed_start_ind + disc_no * 8] + format_embedded_discourse(disc.discourse_text) + embedded[fixed_end_ind + disc_no * 8:]
        
    in_discourse = False
    for token in embedded.split():
        if token.startswith("<DS>"):
            out_lines.append(f"{token.removeprefix('<DS>')} DS\n")
            in_discourse = True
            continue

        if token.endswith("<DE>") or token.endswith("<DE>."):
            out_lines.append(f"{token.replace('<DE>', '')} DE\n")
            in_discourse = False
            continue

        if in_discourse:
            out_lines.append(f"{token} DB\n")
        else:
            out_lines.append(f"{token} ND\n")


    if ind == 10:
        break

    out_lines.append("\n")

with open("../data/test_train.txt", "w") as f:
    f.writelines(out_lines)

Phones Modern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop. All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it. When people have phones they know about certain apps that they have. Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation. Driving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems. That's why there's a thing that's called no texting while driving. That's a really important thing to remember. Some 